In [1]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
def getVariants(y):
    variants = []
    for i in range(0, np.amax(y) + 1):
        variants.append([])

    for i in range(0, y.shape[0]):
        variants[y[i]].append(i)

    return variants

In [3]:
from sklearn.linear_model import LogisticRegression
class OneVsOne(object):
    def __init__(self):
        self.models = []
        
    def fit(self, dataSet, y):
        X = dataSet['data']
        setosa_y = (dataSet['target']==0).astype(int).reshape(150,1)
        versicolor_y = (dataSet['target']==1).astype(int).reshape(150,1)
        virginica_y = (dataSet['target']==2).astype(int).reshape(150,1)
        
        modelAvBC = LogisticRegression(C=10**10)
        modelBvAC = LogisticRegression(C=10**10)
        modelCvAB = LogisticRegression(C=10**10)

        modelAvBC.fit(X, setosa_y.ravel())
        modelBvAC.fit(X, versicolor_y.ravel())
        modelCvAB.fit(X, virginica_y.ravel())
        
        self.models = np.array([modelAvBC, modelBvAC, modelCvAB])
                
    def predict(self,  test_datas):
        def _predict(models, test_data):
            test_data = np.reshape(test_data, (1, -1))
            vote = 0
            debug = []
            for model in models:
                pred = model.predict(test_data)[0]
                debug.append(pred)
                if pred == 1:
                    vote = np.array(np.where(models == model))[0][0]
            print(debug)
            return vote
            
        predict = []
        for data in test_datas:
            predict.append(_predict(self.models, data))
        return np.array(predict)

In [4]:
ovo = OneVsOne()

In [5]:
X = iris['data']
y = (iris['target']).astype(int).reshape(150,1)

In [6]:
ovo.fit(iris, y.ravel())

In [7]:
iris['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [8]:
variants = getVariants(y.ravel())
setosa_y = (iris['target'][variants[0]]).astype(int).reshape(50,1)
versicolor_y = (iris['target'][variants[1]]).astype(int).reshape(50,1)
virginica_y = (iris['target'][variants[2]]).astype(int).reshape(50,1)

setosa_x = X[variants[0]]
versicolor_x = X[variants[1]]
virginica_x = X[variants[2]]

In [9]:
testData = np.concatenate([setosa_x[40:], versicolor_x[40:], virginica_x[40:]])

ovo.predict(testData)

[1, 0, 0]
[1, 1, 0]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[0, 1, 0]
[0, 0, 0]
[0, 1, 0]
[0, 1, 0]
[0, 1, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 1, 0]
[0, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 1, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]


array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])